In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore")

import janestreet
env = janestreet.make_env()

# First look at the data

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')

print(train.head())

# Setting up datasets (don't want to train with no-impact rows, don't want to mess up with clever filling of nans - means can change for half a year!)

In [ ]:
features = [col for col in list(train.columns) if 'feature' in col]
train = train[train['weight'] != 0]
train['action'] = (train['resp'].values > 0).astype(int)
train = train.fillna(-999999)
X_train = train.loc[:, features]
y_train = train.loc[:, 'action']
del train

# Seems that many things there could be changed...

In [ ]:
xgcr = xgb.XGBClassifier(
    n_estimators=1000,
    max_depth=7,
    eta=0.1,
    missing=-999999,
    random_state=29,
    tree_method='gpu_hist',
    subsample=0.95,
    colsample_bytree=0.95,
    eval_metric='error'
)

In [ ]:
%time xgcr.fit(X_train, y_train)

In [ ]:
for (test_df, prediction_df) in env.iter_test():
    X_test = test_df.loc[:, features]
    y_preds = xgcr.predict(X_test)
    prediction_df.action = y_preds
    env.predict(prediction_df)